In [2]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta, date
from collections import defaultdict
import json

In [3]:
df = pd.read_csv('../../data/race_data/2019.csv')

In [4]:
new_times_lap = []
for index, time in df['Lap Time'].iteritems():
    if time.count(':') == 1:
        new_times_lap.append(datetime.strptime(time, '%M:%S.%f'))
    else:
        new_times_lap.append(datetime.strptime(time, '%H:%M:%S.%f'))
df['Lap Time'] = new_times_lap

In [5]:
new_times_s = []
date_index = 100000
for index, time in df['Session Time'].iteritems():
    if time.startswith('24') and time.count(':') > 1:
        date_index = index
        new_time = '01:00' + time[2:]
        new_date = datetime.strptime(new_time, '%d:%H:%M:%S.%f')
        new_date_plus = new_date + timedelta(days=1)
        new_times_s.append(new_date_plus)
    elif index > date_index:
        new_time = '01:00:0' + time
        new_date = datetime.strptime(new_time, '%d:%H:%M:%S.%f')
        new_date_plus = new_date + timedelta(days=1)
        new_times_s.append(new_date_plus)
    elif time.count(':') == 1:
        new_times_s.append(datetime.strptime(time, '%M:%S.%f'))
    else:
        new_times_s.append(datetime.strptime(time, '%H:%M:%S.%f'))
df['Session Time'] = new_times_s

In [6]:
df['Prev Session Time'] = df['Session Time'].subtract(df['Lap Time'])

In [7]:
for i in range(1,14):
    num_str = str(i)
    if i < 10: num_str = '0' + num_str
    num_str = 'S' + num_str
    
    vals = []
    for ind, split_time in df[num_str].iteritems():
        vals.append(timedelta(seconds=split_time))
    df[num_str] = vals
    totals = df[num_str] + df['Prev Session Time']
    
    for j in range(i-1, 0, -1):
        new_num_str = str(j)
        if j < 10: new_num_str = '0' + new_num_str
        new_num_str = 'S' + new_num_str
        totals += df[new_num_str]
    df[num_str+'totals'] = totals

In [8]:
df.to_csv('../../data/2019/2019_mod.csv', index=False, date_format='%Y-%m-%d %H:%M:%S.%f')

In [9]:
section_mod = defaultdict(list)
for index, row in df.iterrows():
    section_mod[row['Driver']].append((1, row['Prev Session Time'].total_seconds(), row['S01totals'].total_seconds(), row.Car, row.SPI))
    section_mod[row['Driver']].append((2, row['S01totals'].total_seconds(), row['S02totals'].total_seconds(), row.Car, row.SPI))
    section_mod[row['Driver']].append((3, row['S02totals'].total_seconds(), row['S03totals'].total_seconds(), row.Car, row.SPI))
    section_mod[row['Driver']].append((4, row['S03totals'].total_seconds(), row['S04totals'].total_seconds(), row.Car, row.SPI))
    section_mod[row['Driver']].append((5, row['S04totals'].total_seconds(), row['S05totals'].total_seconds(), row.Car, row.SPI))
    section_mod[row['Driver']].append((6, row['S05totals'].total_seconds(), row['S06totals'].total_seconds(), row.Car, row.SPI))
    section_mod[row['Driver']].append((7, row['S06totals'].total_seconds(), row['S07totals'].total_seconds(), row.Car, row.SPI))
    section_mod[row['Driver']].append((8, row['S07totals'].total_seconds(), row['S08totals'].total_seconds(), row.Car, row.SPI))
    section_mod[row['Driver']].append((9, row['S08totals'].total_seconds(), row['S09totals'].total_seconds(), row.Car, row.SPI))
    section_mod[row['Driver']].append((10, row['S09totals'].total_seconds(), row['S10totals'].total_seconds(), row.Car, row.SPI))
    section_mod[row['Driver']].append((11, row['S10totals'].total_seconds(), row['S11totals'].total_seconds(), row.Car, row.SPI))
    section_mod[row['Driver']].append((12, row['S11totals'].total_seconds(), row['S12totals'].total_seconds(), row.Car, row.SPI))
    section_mod[row['Driver']].append((13, row['S12totals'].total_seconds(), row['S13totals'].total_seconds(), row.Car, row.SPI))

In [10]:
MAXTIME = df['S13totals'].max().total_seconds()
def roundTo(x, base=20):
    return base * round(x/base)
MAXTIME = roundTo(MAXTIME, 20)
print(MAXTIME)

79300


In [11]:
pit_times = {}
for car in list(pd.unique(df.Car)):
    df_car = df.loc[(df['Car'] == car) & (df['Location'] == 'Pit')]
    pt = list(pd.unique(df_car['Session Time']))
    d_zero = np.datetime64('1900-01-01T00:00:00')
    pt = [(d - d_zero) / np.timedelta64(1, 's') for d in pt]
    pt = [0] + pt
    pit_times[car] = pt

In [28]:
def getLastPit(car, time):
    lp = max([t for t in pit_times[car] if t <= time])
    return (time - lp) / 60

avg_pit = {}
for k, v in pit_times.items():
    diffs = [j-i for i, j in zip(v[:-1], v[1:])]
    avg_pit[k] = np.average(diffs) / 60
    
aplist = [k for k,v in sorted(avg_pit.items(), key=lambda t: t[1], reverse=True)]
pos_dict = {}
for i in range(len(aplist)):
    pos_dict[aplist[i]] = i+1

print(pos_dict[99], avg_pit[99], getLastPit(99, 5860), np.divide(pit_times[99],60), 5860 / 60)
print(pos_dict[24], avg_pit[24], getLastPit(24, 9780), np.divide([t for t in pit_times[24] if t < 9780],60), 9780 / 60)
print(pos_dict[50], avg_pit[50], getLastPit(50, 13800), np.divide([t for t in pit_times[50] if t < 13800],60), 13800 / 60)
print(pos_dict[52], avg_pit[52], getLastPit(52, 20460), np.divide([t for t in pit_times[52] if t < 20460],60), 20460 / 60)
print(pos_dict[77], avg_pit[77], getLastPit(77, 24200), np.divide([t for t in pit_times[77] if t < 24200],60), 24200 / 60)
print(pos_dict[47], avg_pit[47], getLastPit(47, 28920), np.divide([t for t in pit_times[47] if t < 28920],60), 28920 / 60)
print(pos_dict[50], avg_pit[50], getLastPit(50, 36340), np.divide([t for t in pit_times[50] if t < 36340],60), 36340 / 60)
print(pos_dict[3], avg_pit[3], getLastPit(3, 42960), np.divide([t for t in pit_times[3] if t < 42960],60), 42960 / 60)
print(pos_dict[55], avg_pit[55], getLastPit(55, 48500), np.divide([t for t in pit_times[55] if t < 48500],60), 48500 / 60)

47 23.72078888888889 26.504299999999997 [ 0.         11.18738333 20.49768333 71.16236667] 97.66666666666667
21 49.29854423076924 1.5247333333333397 [  0.          49.11948333  78.42086667 112.88418333 161.47526667] 163.0
38 37.754215196078434 4.884599999999986 [  0.          31.5821      65.4436     105.37155    146.1744
 169.10181667 210.87715    225.1154    ] 230.0
31 40.78136666666667 25.191649999999996 [  0.          30.9099      68.6341     106.25866667 112.0122
 115.8292     150.35223333 170.75578333 210.45428333 239.25991667
 279.51465    315.80835   ] 341.0
43 35.62964242424242 11.407266666666692 [  0.          31.45018333  66.74295    105.79661667 146.38715
 169.3136     212.80665    238.52173333 278.38268333 313.59298333
 350.06413333 391.92606667] 403.3333333333333
3 60.854992857142854 10.717083333333358 [  0.          49.4018      52.81271667  75.16556667 108.54556667
 161.74493333 220.96283333 226.75726667 291.38925    353.66233333
 414.73416667 471.28291667] 482.0
38 37.7

In [32]:
l

In [12]:
distances = {1: 1876, 2: 1751.33, 3: 1768.67, 4: 2352, 5: 1488, 6: 1475, 7: 1427, 
             8: 1428, 9: 1457, 10: 1484, 11: 1283, 12: 120, 13: 886.83}

num_drivers = defaultdict(lambda: defaultdict(list))
concentration = defaultdict(lambda: defaultdict(float))
last_pit = {}
avg_speed = {}
for t in range(20, MAXTIME, 20):
    cars = []
    speeds = []
    for driver in section_mod:
        for interval in section_mod[driver]:
            if interval[1] <= t <= interval[2]:
                num_drivers[t][interval[0]].append((interval[1], interval[2]))
                cars.append(interval[3])
                speeds.append(interval[4])
                break
    for section in num_drivers[t]:
        concentration[t][section] = (len(num_drivers[t][section]) / distances[section]) * 1000
        # cars per 1000 feet
    
    all_times = []
    for car in cars:
        lp = max(filter(lambda pt: pt < t, pit_times[car]))
        all_times.append(t - lp)
    if not 66260<=t<=66460:
        last_pit[t] = np.max(all_times) / 60
    avg_speed[t] = np.average(speeds)

C:\Users\dithu\anaconda3\lib\site-packages\numpy\lib\function_base.py:380: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
C:\Users\dithu\anaconda3\lib\site-packages\numpy\core\_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [13]:
with open('../../data/2019/19_num_drivers.json', 'w') as outfile:
    json.dump(num_drivers, outfile)
with open('../../data/2019/19_concentrations.csv', 'w') as f:
    f.write('Time,Sector,Concentration\n')
    for time in concentration.keys():
        for sector in range(1,14):
            f.write("%s,%s,%s\n"%(time, sector, concentration[time].get(sector, 0)))

In [14]:
with open('../../data/2019/19_time_since_pit.csv', 'w') as f:
    f.write('Time,Last Pit\n')
    for time, lp in last_pit.items():
        f.write("%s,%s\n"%(time, lp))

In [15]:
with open('../../data/2019/19_avg_speed.csv', 'w') as f:
    f.write('Time,Speed\n')
    for time, speed in avg_speed.items():
        f.write("%s,%s\n"%(time, speed))